In [1]:
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import keras

In [2]:
df = pd.read_csv("train_for_predict_adr.csv")
df = df.drop(["date"], axis=1)
df.head()

,hotel,is_canceled,lead_time,arrival_date_week_number,stays_in_weekend_nights,stays_in_week_nights,adults,children,is_repeated_guest,previous_cancellations,...,distribution_channel_GDS,distribution_channel_TA/TO,distribution_channel_Undefined,deposit_type_No Deposit,deposit_type_Non Refund,deposit_type_Refundable,customer_type_Contract,customer_type_Group,customer_type_Transient,customer_type_Transient-Party
0,1,0,257,27,0,2,1,0.0,0,0,...,0,1,0,1,0,0,0,0,1,0
1,1,0,257,27,0,2,2,0.0,0,0,...,0,1,0,1,0,0,0,0,1,0
2,1,0,257,27,0,2,2,0.0,0,0,...,0,1,0,1,0,0,0,0,1,0
3,1,0,257,27,0,2,2,0.0,0,0,...,0,1,0,1,0,0,0,0,1,0
4,1,0,257,27,0,2,2,0.0,0,0,...,0,1,0,1,0,0,0,0,1,0


In [3]:
X = df.drop(["adr"], axis=1).values
y = df["adr"].values

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1126)

In [57]:
earlystop = EarlyStopping(monitor='val_loss', patience=10)
reducelr = ReduceLROnPlateau(monitor='loss', factor=0.2, patience=5, min_lr=0.001)

In [58]:
model = Sequential()
model.add(Dense(256, input_dim=X_train.shape[1], kernel_initializer='glorot_uniform', activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(512, kernel_initializer='glorot_uniform', activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1024, kernel_initializer='glorot_uniform', activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(2048, kernel_initializer='glorot_uniform', activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(5096, kernel_initializer='glorot_uniform', activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(5096, kernel_initializer='glorot_uniform', activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(2048, kernel_initializer='glorot_uniform', activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1024, kernel_initializer='glorot_uniform', activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(512, kernel_initializer='glorot_uniform', activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(512, kernel_initializer='glorot_uniform', activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(256, kernel_initializer='glorot_uniform', activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, kernel_initializer='glorot_uniform', activation='linear'))

model.compile(loss='mean_squared_error', optimizer='adamax')

In [59]:
model.fit(X_train, y_train, epochs=500, batch_size=128, validation_split=0.2, callbacks=[earlystop, reducelr])

Epoch 1/500
286/286 [==============================] - 3s 9ms/step - loss: 4246.7773 - val_loss: 3123.7178
Epoch 2/500
286/286 [==============================] - 3s 9ms/step - loss: 1944.9109 - val_loss: 2775.7524
Epoch 3/500
286/286 [==============================] - 3s 9ms/step - loss: 1606.0081 - val_loss: 2664.8997
Epoch 4/500
286/286 [==============================] - 3s 9ms/step - loss: 1429.1920 - val_loss: 2696.8582
Epoch 5/500
286/286 [==============================] - 3s 9ms/step - loss: 1352.6392 - val_loss: 2462.3010
Epoch 6/500
286/286 [==============================] - 3s 9ms/step - loss: 1279.0608 - val_loss: 2772.3628
Epoch 7/500
286/286 [==============================] - 3s 9ms/step - loss: 1252.5137 - val_loss: 2731.0459
Epoch 8/500
286/286 [==============================] - 3s 9ms/step - loss: 1206.4325 - val_loss: 2513.8428
Epoch 9/500
286/286 [==============================] - 3s 9ms/step - loss: 1141.0723 - val_loss: 1941.6672
Epoch 10/500
286/286 [===============

In [60]:
model.evaluate(X_test, y_test)

358/358 [==============================] - 0s 1ms/step - loss: 414.1227


414.12274169921875

In [61]:
model.save("adr_model.h5")

In [62]:
model.summary()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_208 (Dense)            (None, 256)               20480     
_________________________________________________________________
dropout_190 (Dropout)        (None, 256)               0         
_________________________________________________________________
dense_209 (Dense)            (None, 512)               131584    
_________________________________________________________________
dropout_191 (Dropout)        (None, 512)               0         
_________________________________________________________________
dense_210 (Dense)            (None, 1024)              525312    
_________________________________________________________________
dropout_192 (Dropout)        (None, 1024)              0         
_________________________________________________________________
dense_211 (Dense)            (None, 2048)            

In [4]:
model = keras.models.load_model("adr_model.h5")

In [7]:
y_pred =pd.DataFrame(model.predict(X), columns=["adr"])

In [9]:
y_pred.to_csv("train_adr_nn.csv", index=False)